In [19]:
import pandas as pd
import numpy as np
from CVIX import (filter_contract,cal_forward_price,cal_forward_price)

In [20]:
opt_data = pd.read_csv('opt_data.csv',index_col=[0])
df_rate = pd.read_csv('shibor_df.csv',index_col=0)

In [18]:
maturity, df_jy, df_cjy = filter_contract(opt_data)

In [ ]:
cal_forward_price(maturity['jy'], rf_rate=rf_rate_jy, df=df_jy)

In [21]:
opt_data

,date,close,contract_type,exercise_price,maturity
0,2015-02-09,0.1826,call,2.200,0.120548
656,2015-02-09,0.1614,put,2.350,0.197260
609,2015-02-09,0.1327,put,2.300,0.197260
562,2015-02-09,0.1111,put,2.250,0.197260
515,2015-02-09,0.0880,put,2.200,0.197260
...,...,...,...,...,...
213693,2022-05-27,0.2560,put,3.000,0.071233
220175,2022-05-27,0.2497,call,2.500,0.071233
221944,2022-05-27,0.1185,call,2.900,0.589041
208030,2022-05-27,0.4207,put,3.159,0.071233
